# Data collection
----------

**NOTE:** Data collection process was adapted from a reddit api tutorial in class

In [6]:


# !pip install praw
# !pip install psaw
import praw
from psaw import PushshiftAPI


import pandas as pd
import time

In [3]:
# Using praw to confirm that credentials are working and ...
#  that I am communicating with Reddit correctly

reddit = praw.Reddit(client_id = 'secret',                  # removed post project
                     client_secret = 'extra secret',        # removed post project
                     user_agent = 'DIY Crafts',             # app name
                     username = 'heraticticboom93',         # username
                     password = 'super secret')             # removed post project


# Defining my subreddits for praw
diy = reddit.subreddit('DIY')
crafts = reddit.subreddit('crafts')

# Collecting posts from reddit with praw
diy_new = diy.new(limit = 100)
crafts_new = crafts.new(limit = 100)

# Creating a dictionary per subreddit to read through test collection
diy_dict = { "title":[],
                "score":[],
                "comms_num": [],
                "body":[]}

crafts_dict = { "title":[],
                "score":[],
                "comms_num": [],
                "body":[]}

# Building and adding to those dictionaries
for item in diy_new:
    diy_dict["title"].append(item.title)
    diy_dict["score"].append(item.score)
    diy_dict["comms_num"].append(item.num_comments)
    diy_dict["body"].append(item.selftext)

# sleep between page requests
time.sleep(3)

for item in crafts_new:
    crafts_dict["title"].append(item.title)
    crafts_dict["score"].append(item.score)
    crafts_dict["comms_num"].append(item.num_comments)
    crafts_dict["body"].append(item.selftext)

In [1]:
# collect total of 15,000 rows for analysis
# collecting all data via psaw api
api = PushshiftAPI(reddit)

# creating dictionary to structure the information
crafts_dict = { "title":[],
                "score":[],
                "comms_num": [],
                "body":[]}

counter = 0             # counts the number of posts collected
sleep_counter = 0       # timer to not overload the server
for item in api.search_submissions(subreddit='crafts', limit = 7_500):
    crafts_dict['title'].append(item.title)
    crafts_dict['score'].append(item.score)
    crafts_dict['comms_num'].append(item.num_comments)
    crafts_dict['body'].append(item.selftext)
    
    counter += 1
    sleep_counter += 1
    if sleep_counter > 99:
        print(f'collected {counter} \'Crafts\' posts')
        time.sleep(3)
        sleep_counter = 0
        
        
# collecting crafts in the same manner as DIY
diy_dict = { "title":[],
                "score":[],
                "comms_num": [],
                "body":[]}
counter = 0
sleep_counter = 0
for item in api.search_submissions(subreddit='diy', limit = 7_500):
    diy_dict['title'].append(item.title)
    diy_dict['score'].append(item.score)
    diy_dict['comms_num'].append(item.num_comments)
    diy_dict['body'].append(item.selftext)
    counter += 1
    sleep_counter += 1
    if sleep_counter > 99:
        print(f'collected {counter} \'DIY\' posts')
        time.sleep(3)
        sleep_counter = 0

## Cleaning emojis
_________________
Cleaning the emoji's before saving the information to a csv will guarantee an easier cleaning process. 

The function was addapted from [stack overflow](https://stackoverflow.com/questions/33404752/removing-emojis-from-a-string-in-python)

In [34]:
# Clean the emojis from the dictionaries to make cleaning and EDA a lot smoother

def deEmojify(inputString):
    return string.encode('ascii', 'ignore').decode('ascii')


for i, string in enumerate(diy_dict['title']):
    diy_dict['title'][i] = deEmojify(string)
for i, string in enumerate(crafts_dict['title']):
    crafts_dict['title'][i] = deEmojify(string)

    
for i, string in enumerate(diy_dict['body']):
    try:
        diy_dict['body'][i] = deEmojify(string)
    except:
        pass

for i, string in enumerate(crafts_dict['body']):
    try:
        crafts_dict['body'][i] = deEmojify(string)
    except:
        pass

,title,score,comms_num,body
0,"Badly damaged exterior wall studs, is this out...",2,0,[Pictures.](https://imgur.com/a/uafBlta) This ...
1,Need help removing a screw from the back of a ...,1,1,
2,Add second post parallel to existing post,1,0,I have an idea to mount my HD antenna on a pol...
3,rtyuiooopfghjkl,1,0,
4,DIY - recycled vinyl record clock,1,1,


In [38]:
# raw data collection is done, transfering to csv for cleaning
pd.DataFrame(crafts_dict).to_csv('raw data/crafts.csv', index = False)
pd.DataFrame(diy_dict).to_csv('raw data/diy.csv', index = False)

# raw data collection completed and saved